In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
print(os.listdir())

/content
['.config', 'drive', 'sample_data']


In [20]:
urls = ""
fileName = "part2_3"
with open('/content/drive/MyDrive/tesco-urls/' + fileName + '.json', 'r') as file:
    urls = json.load(file)

In [21]:
# URL of the page to scrape
url = "https://www.tesco.ie/groceries/en-IE/products/300112569" #cream crackers URL

# Send a GET request to the URL
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
items = []

for url in urls:
  response = requests.get(url, headers=headers)
  response.encoding = 'utf-8'

  # Parse the page content
  soup = BeautifulSoup(response.text, 'html.parser')
  nutritional_info = {
      "url": url
  }
  try:
      nutritional_info["ProductName"] = soup.select_one('h1').get_text(strip=True)
      nutritional_info["Unit"] = soup.select('thead th')[1].get_text(strip=True)
  except IndexError:
    print(url, "has index out of range")
  except Exception as e:
    print(f"An error occurred for {url}: {e}")


  price_elements = soup.select('.ddsweb-price__container > p')
  # Extract the first and second elements, if available
  try:
    price = price_elements[0].get_text(strip=True) if len(price_elements) > 0 else "Price not found"
    nutritional_info["price"] = price
    price_per_kg = price_elements[1].get_text(strip=True) if len(price_elements) > 1 else "Price per kg not found"
    nutritional_info["price_per_kg"] = price_per_kg
  except IndexError:
    print(url, "has index out of range for price")
  except Exception as e:
    print(f"An error occurred for {url} for price: {e}")
  # Find all target div elements within #accordion-panel-product-description
  content_divs = soup.select('#accordion-panel-product-description > div > div')
  net_contents = None  # Initialize the variable
  for div in content_divs:
      # Check if the immediate previous sibling is an h3 with the text "Net Contents"
      prev_sibling = div.find_previous_sibling()
      if prev_sibling and prev_sibling.name == "h3" and prev_sibling.get_text(strip=True) == "Net Contents":
          # Found the correct div
          net_contents = div.get_text(strip=True)
          break  # Exit loop once the desired div is found

  nutritional_info["NetContents"] = net_contents
  unique_columns = set(); # just to list out the unique columns
  row_elements = soup.select("tbody tr")
  try:
    for row_element in row_elements:
      nutrient = row_element.select_one("th").get_text(strip=True)
      value = row_element.select_one("td").get_text(strip=True)
      if value == '-' or value =="":
        continue
      nutrient = nutrient.replace('-', '_').lower()
      nutritional_info[nutrient] = value
      unique_columns.add(nutrient)
  except AttributeError:
    print(f"AttributeError: 'tbody tr' not found for {url}")
  items.append(nutritional_info)

df = pd.DataFrame(items)
df.to_csv('/content/drive/MyDrive/tesco-urls/' + fileName + '.csv', index=False)


# print("Product Name:", title)
# print("Net Contents:", net_contents)
# print("Unit:", unit)
# print("Nutritional Information:", nutritional_info)
# print("Unique Columns:", unique_columns)

https://www.tesco.ie/groceries/en-IE/products/275156636 has index out of range
https://www.tesco.ie/groceries/en-IE/products/252807478 has index out of range
https://www.tesco.ie/groceries/en-IE/products/261058238 has index out of range
https://www.tesco.ie/groceries/en-IE/products/296735680 has index out of range
https://www.tesco.ie/groceries/en-IE/products/296704688 has index out of range
https://www.tesco.ie/groceries/en-IE/products/316536847 has index out of range
https://www.tesco.ie/groceries/en-IE/products/276067803 has index out of range
https://www.tesco.ie/groceries/en-IE/products/314316734 has index out of range
https://www.tesco.ie/groceries/en-IE/products/295163384 has index out of range
https://www.tesco.ie/groceries/en-IE/products/313844907 has index out of range
https://www.tesco.ie/groceries/en-IE/products/315463577 has index out of range
https://www.tesco.ie/groceries/en-IE/products/304724456 has index out of range
https://www.tesco.ie/groceries/en-IE/products/311216

In [ ]:
len(items)

1

In [ ]:
# df = pd.DataFrame(items)
# df.to_csv('first_scrape.csv', index=False)